We're almost there:
- We loaded the documents
- We've split them in to relevant parts (splitters)
- We've had the vector database calculate the embeddings and store the related parts

Related Langchain example: https://python.langchain.com/docs/use_cases/question_answering/

In [1]:
%pip install langchain openai chromadb

Note: you may need to restart the kernel to use updated packages.


More of the boilerplate we prepared earlier.

In [2]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI


from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# We load the texts
from langchain.text_splitter import MarkdownHeaderTextSplitter

history_raw_text = ""
    # This is a long document we can split up.
with open("data/history.md") as f:
    history_raw_text = f.read()
    
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
raw_documents = md_splitter.split_text(history_raw_text)

from pprint import pprint
pprint(raw_documents)

# Set the embeddings function
embeddings_model = OpenAIEmbeddings()

# Resetting chromadb just in case
import chromadb
from chromadb import Settings
client = chromadb.Client(settings=Settings(allow_reset=True))
client.reset()

db = Chroma.from_documents(raw_documents, embeddings_model)


[Document(page_content='Devopsdays is a worldwide series of technical conferences covering topics of software development, IT infrastructure operations, and the intersection between them. Each event is run by volunteers from the local area.  \nMost devopsdays events feature a combination of curated talks (see open Calls for Proposals) and self organized open space content. Topics often include automation, testing, security, and organizational culture.', metadata={'Header 1': 'A history lesson on Devops', 'Header 2': 'Devopsdays'}),
 Document(page_content='The first devopsdays was held in Ghent, Belgium in 2009. Since then, devopsdays events have multiplied, and if there isn’t one in your city, check out the information about organizing one yourself!', metadata={'Header 1': 'A history lesson on Devops', 'Header 2': 'Devopsdays', 'Header 3': 'History'}),
 Document(page_content='The devopsdays global core team guides local organizers in hosting their own devopsdays events worldwide. Activ

Now this is the step we put it all together by using the RetrievalQA chain.
This combines the vectordatabase , the prompt , the llm to return answer.


In [3]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
from _lessonshelper.pretty_print_callback_handler import PrettyPrintCallbackHandler

pretty_print_callback = PrettyPrintCallbackHandler()
llm.callbacks = [pretty_print_callback]

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=db.as_retriever(), return_source_documents=True
)


For comparison let's first do our query through the regular llm call.
You see it returns 4 authors of the Devops Handbook.

In [4]:

query = "Who were the people involved in writing the devops Handbook ? If there are multiple authors return them all. Return the result as json and use the field firstname and lastname"


# First via the llm
from langchain.schema import (
    HumanMessage,
)

answer = llm([HumanMessage(content=query)])
pprint(answer)


[chat_model][start] - prompts : Who were the people involved in writing the devops Handbook ? If there are multiple authors return them all. Return the result as json and use the field firstname and lastname

[llm][end] - generation The authors of "The DevOps Handbook" are Gene Kim, Jez Humble, Patrick Debois, and John Willis. Here is the information in JSON format:

[
  {
    "firstname": "Gene",
    "lastname": "Kim"
  },
  {
    "firstname": "Jez",
    "lastname": "Humble"
  },
  {
    "firstname": "Patrick",
    "lastname": "Debois"
  },
  {
    "firstname": "John",
    "lastname": "Willis"
  }
]
AIMessage(content='The authors of "The DevOps Handbook" are Gene Kim, Jez Humble, Patrick Debois, and John Willis. Here is the information in JSON format:\n\n[\n  {\n    "firstname": "Gene",\n    "lastname": "Kim"\n  },\n  {\n    "firstname": "Jez",\n    "lastname": "Humble"\n  },\n  {\n    "firstname": "Patrick",\n    "lastname": "Debois"\n  },\n  {\n    "firstname": "John",\n    "lastnam

Now with the retrieval chain we see it returns 5 authors. The extra one being John Allspaw who wrote the foreword.

In [5]:

# next via the llm via Retrieval Augmented Generation
answer = qa_chain({"query": query})
pprint(answer)


[chat_model][start] - prompts : Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
The Devops Handbook was written by the following authors: Gene Kim, Jez Humble , John Willis , Patrick Debois and John Allspaw.  
Gene Kim is a multiple award-winning entrepreneur, the founder and former CTO of Tripwire and a researcher. He is passionate about IT operations, security and compliance, and how IT organizations successfully transform from "good to great." He lives in Portland, Oregon.  
Jez Humble is an award-winning author and researcher on software who has spent his career tinkering with code, infrastructure, and product development in organizations of varying sizes across three continents. He works at 18F, teaches at UC Berkeley, and is co-founder of DevOps Research and Assessment LLC.  
Patrick Debois is an independent IT-consultant who is bridging the gap between

This combination of RAG is one of the primary patterns people are using LLMs with their own data.